Rewrite the code to make it look more professional

In [2]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, cycle, product, islice, chain
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional
from sys         import maxsize

import re
import ast
import operator

import numpy as np

In [3]:
def read_data(input: str, parser=str, sep='\n', testing=False) -> list:
    if testing:
        sections = input.split(sep)
    else:
        sections = open(input).read().split(sep)
    return [parser(section) for section in sections]

In [4]:
test_string = """Player 1:
9
2
6
3
1

Player 2:
5
8
4
7
10"""
test_ins = read_data(test_string, sep="\n\n", testing=True)

In [5]:
test_ins

['Player 1:\n9\n2\n6\n3\n1', 'Player 2:\n5\n8\n4\n7\n10']

In [57]:
def run_part1(ins: List[str]) -> int:
    player_1, player_2 = [deque(int(x) for x in re.sub(r"Player \d+:\n", " ", card_group).split()) for card_group in ins]
    while player_1 and player_2:
        plays = list(x.popleft() for x in (player_1, player_2))
        sorted_plays = sorted(plays, reverse=True)
        player_1.extend(sorted_plays) if plays[0] > plays[1] else player_2.extend(sorted_plays)
        # print(player_1, player_2)

    winner = player_2 if len(player_1) == 0 else player_1
    return sum(idx*val for idx, val in enumerate(reversed(winner), 1))

In [58]:
run_part1(test_ins)

306

Part I  

Play the small crab in a game of Combat using the two decks you just dealt. What is the winning player's score?

In [59]:
real_ins = read_data("input.txt", sep="\n\n")
run_part1(real_ins)

35005

Part II

After updating rules 8 and 11, how many messages completely match rule 0?

In [160]:
# check if there was a prev round with exactly the same cards
def run_part2(ins: List[str]) -> int:

    # set representing game history: set((List[int], List[int]))
    history = set()
    def iterate_part2(player_1: List[int], player_2: List[int], game_history: set):
        # print(player_1, player_2)
        # print("game history")
        # print(game_history)
        # if not player_1 or not player_2:
            
        #     return player_1, player_2
        while player_1 and player_2:
            # check and add seen combinations
            if (tuple(player_1), tuple(player_2)) in game_history:
                # player 1 wins
                # print("recursive error")
                # player_1.rotate(-1)
                # player_1.extend(player_2)
                return player_1, deque()
            game_history.add(tuple(map(tuple, (player_1, player_2))))

            play_1, play_2 = list(x.popleft() for x in (player_1, player_2))
            if play_1 <= len(player_1) and play_2 <= len(player_2):
                # print("going into recursive game")
                # player recursive game with new history
                sub_player_1, sub_player_2 = iterate_part2(
                                            deque(islice(player_1, 0, play_1)), 
                                            deque(islice(player_2, 0, play_2)),
                                            set()
                                            )
                if not sub_player_1:
                    player_2.extend([play_2, play_1])
                else:
                    player_1.extend([play_1, play_2])
            else:
                # play usual game
                # print("playing usual game")
                player_1.extend([play_1, play_2]) if play_1 > play_2 else player_2.extend([play_2, play_1])
        
        # print("ending game")
        return player_1, player_2
        
    player_1, player_2 = [deque(int(x) for x in re.sub(r"Player \d+:\n", " ", card_group).split()) for card_group in ins]
    # print("starting iters")
    player_1, player_2 = iterate_part2(player_1, player_2, history)

    # print("all iters ended")
    winner = player_2 if not player_1 else player_1
    return sum(idx*val for idx, val in enumerate(reversed(winner), 1))

   

In [161]:
run_part2(test_ins)

291

In [162]:
test_string2 = """Player 1:
43
19

Player 2:
2
29
14
"""
test_ins2 = read_data(test_string2, sep="\n\n", testing=True)
run_part2(test_ins2)

105

In [163]:
run_part2(real_ins)

32751

In [164]:
print([deque(int(x) for x in re.sub(r"Player \d+:\n", " ", card_group).split()) for card_group in real_ins])

[deque([21, 22, 33, 29, 43, 35, 8, 30, 50, 44, 9, 42, 45, 16, 12, 4, 15, 27, 20, 31, 25, 47, 5, 24, 19]), deque([3, 40, 37, 14, 1, 13, 49, 41, 28, 48, 18, 7, 23, 38, 32, 34, 46, 39, 17, 2, 11, 6, 10, 36, 26])]


In [165]:
def recursive_combat(deals: Deals) -> Deals:
    "A game of Recursive Combat."
    deals = mapt(deque, deals)
    previously = set()
    while all(deals):
        if seen(deals, previously):
            return (deals[0], ())
        topcards = mapt(deque.popleft, deals)
        if all(len(deals[p]) >= topcards[p] for p in (0, 1)):
            deals2 = [tuple(deals[p])[:topcards[p]] for p in (0, 1)]
            result = recursive_combat(deals2)
            winner = 0 if result[0] else 1
        else:
            winner = 0 if topcards[0] > topcards[1] else 1
        deals[winner].extend([topcards[winner], topcards[1 - winner]])
    return deals

def seen(deals, previously) -> bool:
    "Return True if we have seen this pair of deals previously; else just remember it."
    hasht = mapt(tuple, deals)
    if hasht in previously:
        return True
    else:
        previously.add(hasht)
        return False

def mapt(fn, *args): 
    "Do map(fn, *args) and make the result a tuple."
    return tuple(map(fn, *args))

In [166]:
deals = [deque([21, 22, 33, 29, 43, 35, 8, 30, 50, 44, 9, 42, 45, 16, 12, 4, 15, 27, 20, 31, 25, 47, 5, 24, 19]), deque([3, 40, 37, 14, 1, 13, 49, 41, 28, 48, 18, 7, 23, 38, 32, 34, 46, 39, 17, 2, 11, 6, 10, 36, 26])]

recursive_combat(deals)


(deque([18, 3, 44, 39, 36, 17, 38, 34, 7, 5, 50, 15, 22, 2, 49, 23, 28, 8, 47, 40, 43, 25, 30, 1, 42, 37, 13, 11, 48, 16, 24, 14, 32, 20, 10, 4, 46, 19, 26, 6, 45, 29, 41, 12, 35, 9, 33, 27, 31, 21]), deque([]))